In [2]:
import json

In [11]:
with open('gg2020.json', encoding='utf8') as f:
    lines = f.readlines()

gg_data = []
for line in lines:
    gg_data.append(json.loads(line))
    

In [12]:
gg_data[0]

{'_id': {'$oid': '5e17fa7e174f9084edc115f2'},
 'user_id': '2414944424',
 'text': '• Priyanka Chopra and Nick Jonas at the 77th Annual Golden Globe Awards present the award for Best TV series for a musical or comedy • #goldenglobes2020 @ Golden Globes https://t.co/fivnKsE9Ib',
 'created_at': '2020-01-06T23:59:57',
 'user': 'COttavianoNYC',
 'id': '1214335827507122176'}